<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_pytorch/blob/main/HW_04_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ДЗ 4 Pytorch

## Задание 1

Необходимо доработать обучение нейросети, что мы разбирали на уроке.(Посмотрите чего не хватает в процессе обучения и подготовки данных)

### Базовый вариант

In [ ]:
pip install segmentation_models_pytorch

     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 245kB 8.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60963 sha256=8e239dfed85d3a0a67e39a7bc8e92d6866615c4631e3fe7bd00b92aee91c21a6
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12420 sha256=b2b9330abf39bf846de8502cd529dca52a62565b85fcf68be731e057f720a55d
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch


In [ ]:
import segmentation_models_pytorch as smp

In [5]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vitalygladyshev","key":"3353367a1e22b48542ed3c3946bdf3e4"}'}

In [7]:
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle/

total 16
drwxr-xr-x 2 root root 4096 Dec 15 07:45 .
drwx------ 1 root root 4096 Dec 15 07:44 ..
-rw------- 1 root root   71 Dec 15 07:44 kaggle.json


In [9]:
!kaggle datasets list -s lyft-udacity-challenge

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
kumaresanmanickavelu/lyft-udacity-challenge               Semantic Segmentation for Self Driving Cars          5GB  2018-05-18 05:59:42           1996  
morrisb/semantic-segmentation-with-carla-and-tpus         Semantic Segmentation With CARLA And TPUs            3GB  2020-10-18 10:11:51             16  
austenmy/udacity-enrollments                              Udacity Enrollments                                 10KB  2018-04-06 00:54:43             91  
pbdanny/udacity-project-submissions                       Udacity Project Submission                          24KB  2018-05-10 11:41:36             45  
samtyagi/audacity-ab-testing                              audacity ab testing     

In [12]:
!kaggle datasets download -d kumaresanmanickavelu/lyft-udacity-challenge
!ls

100% 5.09G/5.11G [01:12<00:00, 95.1MB/s]
100% 5.11G/5.11G [01:12<00:00, 75.9MB/s]
lyft-udacity-challenge.zip  sample_data


In [14]:
from zipfile import ZipFile

In [15]:
zip_file = ZipFile('lyft-udacity-challenge.zip')
zip_file.extractall("./lyft-udacity-challenge")

In [20]:
!path = "lyft-udacity-challenge"
!ls -ls $path

/bin/bash: path: command not found
total 40
4 drwxr-xr-x 3 root root 4096 Dec 15 08:09 dataa
4 drwxr-xr-x 3 root root 4096 Dec 15 08:08 dataA
4 drwxr-xr-x 3 root root 4096 Dec 15 08:09 datab
4 drwxr-xr-x 3 root root 4096 Dec 15 08:08 dataB
4 drwxr-xr-x 3 root root 4096 Dec 15 08:09 datac
4 drwxr-xr-x 3 root root 4096 Dec 15 08:08 dataC
4 drwxr-xr-x 3 root root 4096 Dec 15 08:09 datad
4 drwxr-xr-x 3 root root 4096 Dec 15 08:08 dataD
4 drwxr-xr-x 3 root root 4096 Dec 15 08:09 datae
4 drwxr-xr-x 3 root root 4096 Dec 15 08:08 dataE


In [21]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [22]:
labels = ['Unlabeled','Building','Fence','Other',
          'Pedestrian', 'Pole', 'Roadline', 'Road',
          'Sidewalk', 'Vegetation', 'Car','Wall',
          'Traffic sign']

In [24]:
path = "lyft-udacity-challenge"

In [25]:
cameraRGB = []
cameraSeg = []
for root, dirs, files in os.walk(path):
    for name in files:
        f = os.path.join(root, name)
        if 'CameraRGB' in f:
            cameraRGB.append(f)
        elif 'CameraSeg' in f:
            cameraSeg.append(f)
        else:
            break

In [28]:
cameraRGB[-1], cameraSeg[-1]

('lyft-udacity-challenge/datae/dataE/CameraRGB/06_00_079.png',
 'lyft-udacity-challenge/datae/dataE/CameraSeg/06_00_079.png')

In [32]:
df = pd.DataFrame({'cameraRGB': cameraRGB, 'cameraSeg': cameraSeg})
# Отсортируем  датафрейм по значениям
df.sort_values(by='cameraRGB',inplace=True)
# Используем функцию,
# лагодаря которой индексация значений 
# будет начинаться с 0.
df.reset_index(drop=True, inplace=True)
# Выведем первые ять значений нашего датафрейма
print(df.head(5))

                                           cameraRGB                                          cameraSeg
0  lyft-udacity-challenge/dataA/dataA/CameraRGB/0...  lyft-udacity-challenge/dataA/dataA/CameraSeg/0...
1  lyft-udacity-challenge/dataA/dataA/CameraRGB/0...  lyft-udacity-challenge/dataA/dataA/CameraSeg/0...
2  lyft-udacity-challenge/dataA/dataA/CameraRGB/0...  lyft-udacity-challenge/dataA/dataA/CameraSeg/0...
3  lyft-udacity-challenge/dataA/dataA/CameraRGB/0...  lyft-udacity-challenge/dataA/dataA/CameraSeg/0...
4  lyft-udacity-challenge/dataA/dataA/CameraRGB/0...  lyft-udacity-challenge/dataA/dataA/CameraSeg/0...


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cameraRGB  10000 non-null  object
 1   cameraSeg  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [50]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [34]:
class CustomDatasetFromImages(Dataset):
    def __init__(self, data_info):
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        
        # Разделяем датафрейм на rgb картинки 
        self.image_arr = self.data_info.iloc[:,0]
        # и на сегментированные картинки
        self.label_arr = self.data_info.iloc[:,1]
        
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
    def __getitem__(self, index):
        # Читаем картинку и сразу же представляем ее в виде numpy-массива 
        # размера 600х800 float-значний
        img = np.asarray(Image.open(self.image_arr[index])).astype('float')
        # Нормализуем изображение в значениях [0,1]
        img = torch.as_tensor(img)/255    
        # 1) unsqueeze - меняет размерность img c (600, 800, 3) -> (1, 600, 800, 3),
        # т.е. оборачивает картинку в батч размером в одну картинку
        # 2) permute - меняет местами измерения , т.е. (1, 600, 800, 3) -> (1, 3, 600, 800)
        img = img.unsqueeze(0).permute(0,3,1,2)
        # Мы используем функцию интерполяции для того,
        # чтобы поменять рамерность картинки с 800х600 на 256х256
        img = F.interpolate(input=img, size=256, align_corners=False, mode='bicubic')
        
        # итаем сегментированную картинку и сразу же представляем ее в виде numpy-массива 
        # размера 600х800 float-значний
        lab = np.asarray(plt.imread(self.label_arr[index]))[:,:,0]*255
        
        # Упаковываем ее в pytorch-тензор и оборачиваем ее в батч из одной каринки,
        # но при этом заполняем 13 каналов масками нужных классов
        # Т.е. там, где например класс автомобилей (10 по счету канал) - все пиксели 0 
        # если не принадлежат классу, и 1 если принадлежат 
        x_out = torch.as_tensor(np.where(lab == 0, 255, 0)).unsqueeze(0)
        for i in range(1, 13):
            mask = np.asarray(plt.imread(self.label_arr[index]))[:,:,0]*255
            mask = np.where(mask == i, 255, 0)
            x = torch.as_tensor(mask).unsqueeze(0)
            x_out =  torch.cat((x_out,x),dim=0)    
        x_out = x_out.float()
        
        lab = x_out.unsqueeze(0)
        # делаем ресайз картинки на 256х256
        lab = F.interpolate(input=lab, size=256, mode='nearest')
        
        
        
        return (img.float(), lab.float())

    def __len__(self):
        return self.data_len

In [35]:
from sklearn.model_selection import train_test_split

# 80 % в тренировочную выборку, 20 - в тестовую
X_data, X_test = train_test_split(df, test_size=0.2)
# 20 - в валидационную
X_train, X_valid = train_test_split(X_data, test_size=0.2)

# Упорядочиваем индексацию
X_train.reset_index(drop=True,inplace=True)
X_valid.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

# Оборачиваем каждую выборку в наш кастомный датасет
train_data = CustomDatasetFromImages(X_train)
valid_data = CustomDatasetFromImages(X_valid)
test_data = CustomDatasetFromImages(X_test)

In [45]:
train_data_loader = DataLoader(train_data, batch_size=1, shuffle=True)
valid_data_loader = DataLoader(valid_data, batch_size=1, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [46]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [51]:
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth =1
        num = targets.size(0)
        probs = F.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score =2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score =1 - score.sum() / num
        return score

In [52]:
learning_rate = 0.001
epochs = 5

In [53]:
# создание модели
segmodel = smp.Unet()
segmodel = smp.Unet('resnet34', classes=13, activation='softmax').to(device)

In [54]:
criterion = SoftDiceLoss()
optimizer = torch.optim.Adam(segmodel.parameters(), lr=1e-10)

In [57]:
epoch_losses = []

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    epoch_loss = []
    t = 0
    time1 = time.time()
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device) # .cuda()
        labels = labels.to(device) # .cuda()


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = segmodel(inputs[0])
        loss = criterion(outputs, labels[0,0,:,:,:])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_loss.append(loss.item())
        t+=1
        if not (t+1) % 1000:    # print every 1000 mini-batches
            print(f'Epoch: {epoch}, batchcount: {t}, avg. loss for last 1000 images: {running_loss/1000}')
    time2 = time.time()
    print(f'Epoch {epoch+1}, loss: ',np.mean(epoch_loss),f' time = {time2-time1} sec')
    epoch_losses.append(epoch_loss)

/usr/local/lib/python3.6/dist-packages/segmentation_models_pytorch/base/modules.py:102: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: ignored

### Чего-то не хватает? Добавим PyTorch огня! :)

In [ ]:
pip install git+https://github.com/pytorch/ignite